I will get the dominant colour of the car.

The following code is to get the dominant colour of the image.
The image will be crop to only left with bounding box.
So, the remaining image will contain mostly car.
Therefore, the dominant colour of the precessed image = dominant colour of the car.


In [1]:
import numpy as np
import pandas as pd

import cv2
from sklearn.cluster import KMeans

import webcolors
from os import walk

import os

This is the get the closest colour name in words from the RGB code.

In [2]:
'''
Get closest colour name in words from the RGB code.
'''
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.css3_hex_to_names.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

In [6]:
'''
This will get the dominant colour of the image which can be represented as the dominant colour of the car.
'''

mypath = "color_test"

test_colour = []
test_fname = []
colour_code = []

for (dirpath, dirnames, filenames) in walk(mypath):
    for x in range(0, len(filenames)):
        src_path = os.path.join(mypath, filenames[x])
        image = cv2.imread(src_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        r, c = image.shape[:2]
        out_r = 120
        image = cv2.resize(image, (int(out_r*float(c)/r), out_r))
        
        pixels = image.reshape((-1, 3))
        
        km = KMeans(n_clusters=8)
        km.fit(pixels)
        
        colors = np.asarray(km.cluster_centers_, dtype='uint8')
        percentage = np.asarray(np.unique(km.labels_, return_counts = True)[1], dtype='float32')
        percentage = percentage/pixels.shape[0]

        max_percent = max(percentage)
        max_element = np.where(percentage == max_percent)
        
        actual_name, closest_name = get_colour_name(colors[max_element[0][0]])
        test_colour.append(closest_name)
        test_fname.append(filenames[x])
        colour_code.append(colors[max_element[0][0]])
        
    break
    
print("Done")

Done


In [7]:
'''
This is the result of the dominant colour in words.
'''
submission = pd.DataFrame.from_dict({
    'colour': test_colour,
    'filename': test_fname
})

submission.to_csv('colour_test.csv', index=False)


'''
This is the result of the dominant colour in RGB code.
'''
submission2 = pd.DataFrame.from_dict({
    'rgb_code': colour_code,
    'filename': test_fname
})

submission2.to_csv('rgb_colour.csv', index=False)

'''
This code is tested with 30 images due to the limitation of the computer resources.
'''